In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.26.0 to work with 429test


In [2]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

zombies_model version: 5
	 r2 : 0.8678496063600782
	 r2_adj : 0.8667364261489117


zombies_model version: 4
	 r2 : 0.8678496063600782
	 r2_adj : 0.8667364261489117


zombies_model version: 3
	 r2 : 0.8678496063600782
	 r2_adj : 0.8667364261489117


zombies_model version: 2
	 r2 : 0.8678496063600782
	 r2_adj : 0.8667364261489117


zombies_model version: 1
	 Training context : Script
	 r2 : 0.8678496063600782
	 r2_adj : 0.8667364261489117


diabetes_model version: 2
	 Training context : Parameterized script
	 AUC : 0.8484357430717946
	 Accuracy : 0.774


diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8483203144435048
	 Accuracy : 0.774




In [3]:
model = Model(ws, 'zombies_model', version=2)
print(model.name, 'version', model.version)

zombies_model version 2


In [4]:
import os

folder_name = 'zombies_web_inference'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# Set path for scoring script
script_file = os.path.join(experiment_folder,"score_zombies.py")

zombies_web_inference folder created.


In [5]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('zombies_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    
    y_pred = model.predict(data)
    return json.dumps(y_pred.tolist())

Overwriting ./zombies_web_inference/score_zombies.py


In [6]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')
myenv.add_conda_package('statsmodels')

# Save the environment config as a .yml file
env_file = os.path.join(experiment_folder,"zombies_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())


Saved dependency info in ./zombies_web_inference/zombies_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
- statsmodels
channels:
- anaconda
- conda-forge



In [7]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",entry_script=script_file,conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "zombies-inferencing"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
endpoint = service.scoring_uri
print(endpoint)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-02 01:20:26+00:00 Creating Container Registry if not exists.
2021-05-02 01:20:26+00:00 Registering the environment.
2021-05-02 01:20:28+00:00 Use the existing image.
2021-05-02 01:20:29+00:00 Submitting deployment to compute..
2021-05-02 01:20:35+00:00 Checking the status of deployment zombies-inferencing4..
2021-05-02 01:24:13+00:00 Checking the status of inference endpoint zombies-inferencing4.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://8e300d19-af74-420a-a140-c4c785529f2f.eastus.azurecontainer.io/score


In [11]:
import requests
import json

url = 'http://8e300d19-af74-420a-a140-c4c785529f2f.eastus.azurecontainer.io/score'
payload = '{"data": [[3.258096538021482,-4.605170185988091,8.633374945705647,0.0,0.0,1.0,1.0]]}'
headers = {'content-type': 'application/json'}
response = requests.post(url, data=payload, headers=headers)
print(response.text)

"[2.0227402220525725]"
